In [1]:
from typing import NamedTuple, Sequence, Tuple
import numba as nb
import numpy as np
import json

# Stack allocated array

In [2]:
from numba import types
from numba.extending import intrinsic
from numba.core import cgutils

@intrinsic
def stack_empty(typingctx, size, dtype):
    def impl(context, builder, signature, args):
        ty = context.get_value_type(dtype.dtype)
        ptr = cgutils.alloca_once(builder, ty, size=args[0])
        return ptr
    
    sig = types.CPointer(dtype.dtype)(types.int64,dtype)
    return sig, impl

# Common

In [3]:
FLOAT_TYPE = np.float64

class Point(NamedTuple):
    x: float
    y: float

class Vector(NamedTuple):
    x: float
    y: float

@nb.njit(inline="always")
def cross_product(u: Vector, v: Vector) -> float:
    return u.x * v.y - u.y * v.x

@nb.njit(inline="always")
def dot_product(u: Vector, v: Vector) -> float:
    return u.x * v.x + u.y * v.y

# Sutherland-Hodgman

In [4]:
@nb.njit(inline="always")
def _push(array: np.ndarray, n: int, value: Vector) -> int:
    array[n] = value
    return n + 1

@nb.njit(inline="always")
def _copy(src, dst, n) -> None:
    for i in range(n):
        dst[i] = src[i]

@nb.njit(inline="always")
def _inside(p: Point, r: Point, U: Vector):
    # U: a -> b direction vector
    # p is point r or s
    return U.x * (p.y - r.y) > U.y * (p.x - r.x)

@nb.njit(inline="always")
def _intersection(a: Point, V: Vector, r: Point, N: Vector) -> Tuple[bool, Point]:
    W = Vector(r.x - a.x, r.y - a.y)
    nw = dot_product(N, W)
    nv = dot_product(N, V)
    if nv != 0:
        t = nw / nv
        return True, Point(a.x + t * V.x, a.y + t * V.y)
    else:
        return False, Point(0.0, 0.0)

@nb.njit(inline="always")
def _polygon_area(polygon: Sequence, length: Sequence) -> float:
    area = 0.0
    a = Point(polygon[0][0], polygon[0][1])
    b = Point(polygon[1][0], polygon[1][1])
    U = Vector(b.x - a.x, b.y - a.y)
    for i in range(2, length):
        c = Point(polygon[i][0], polygon[i][1])
        V = Vector(a.x - c.x, a.y - c.y)
        area += abs(cross_product(U, V))
        b = c
        U = V
    return 0.5 * area

def make_allocate(nvertex, ndim):
    size = nvertex * ndim
    
    @nb.njit(inline="always")
    def allocate_empty():
        arr_ptr = stack_empty(size, np.float64)
        arr = nb.carray(arr_ptr, (nvertex, ndim))
        return arr
    
    return allocate_empty

@nb.njit(inline="always")
def clip_polygons(polygon: Sequence, clipper: Sequence) -> float:
    n_output = len(polygon)
    n_clip = len(clipper)
    n_max = n_output + n_clip

    # Create a view on the allocated memory and do something
    subject = allocate_empty()
    output = allocate_empty()
    
    # Copy polygon into output
    _copy(polygon, output, n_output)

    # Grab last point
    r = Point(clipper[n_clip - 1][0], clipper[n_clip - 1][1])
    for i in range(n_clip):
        s = Point(clipper[i][0], clipper[i][1])

        U = Vector(s.x - r.x, s.y - r.y)
        N = Vector(-U.y, U.x)
        if U.x == 0 and U.y == 0:
            continue

        # Copy output into subject
        length = n_output
        _copy(output, subject, length)
        # Reset
        n_output = 0
        # Grab last point
        a = Point(subject[length - 1][0], subject[length - 1][1])
        a_inside = _inside(a, r, U)
        for j in range(length):
            b = Point(subject[j][0], subject[j][1])

            V = Vector(b.x - a.x, b.y - a.y)
            if V.x == 0 and V.y == 0:
                continue

            b_inside = _inside(b, r, U)
            if b_inside:
                if not a_inside:  # out, or on the edge
                    succes, point = _intersection(a, V, r, N)
                    if succes:
                        n_output = _push(output, n_output, point)
                n_output = _push(output, n_output, b)
            elif a_inside:
                succes, point = _intersection(a, V, r, N)
                if succes:
                    n_output = _push(output, n_output, point)
                else:  # Floating point failure
                    b_inside = True  # flip it for consistency, will be set as a
                    n_output = _push(output, n_output, b)  # push b instead

            # Advance to next polygon edge
            a = b
            a_inside = b_inside

        # Exit early in case not enough vertices are left.
        if n_output < 3:
            return 0.0

        # Advance to next clipping edge
        r = s

    area = _polygon_area(output, n_output)
    return area

In [5]:
def _area_of_intersection(a, b):
    n = len(a)
    out = np.zeros(n)
    for i in nb.prange(n):
        t0 = a[i]
        t1 = b[i]
        out[i] = clip_polygons(t0, t1)
    return out

# Read triangles

In [6]:
a = np.load("triangles_a.npy")
b = np.load("triangles_b.npy")

In [7]:
allocate_empty = make_allocate(nvertex=2 * a.shape[1], ndim=a.shape[2])

area_of_intersection = nb.njit(_area_of_intersection)

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/numba.json", "w"))
np.save("answers/numba.npy", area_of_intersection(a[:10_000], b[:10_000]))

The slowest run took 12.00 times longer than the fastest. This could mean that an intermediate result is being cached.
4.12 µs ± 5.47 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.39 µs ± 36.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
12 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
121 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.21 ms ± 5.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.1 ms ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
124 ms ± 963 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.24 s ± 5.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
area_of_intersection = nb.njit(_area_of_intersection, parallel=True)

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/numba-parallel.json", "w"))
np.save("answers/numba-parallel.npy", area_of_intersection(a[:10_000], b[:10_000]))

150 µs ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
136 µs ± 812 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
136 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
157 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
320 µs ± 9.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.12 ms ± 60.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
21.4 ms ± 566 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
208 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
